In [2]:
import pandas as pd

dataframe = pd.read_csv(
    "https://docs.dagster.io/assets/iris.csv",
    names=[
        "sepal_length_cm",
        "sepal_width_cm",
        "petal_length_cm",
        "petal_width_cm",
        "species",
    ],
)

dataframe.values[0:10]

array([[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'],
       [5.4, 3.9, 1.7, 0.4, 'Iris-setosa'],
       [4.6, 3.4, 1.4, 0.3, 'Iris-setosa'],
       [5.0, 3.4, 1.5, 0.2, 'Iris-setosa'],
       [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.1, 1.5, 0.1, 'Iris-setosa']], dtype=object)

In [28]:
from sqlalchemy import create_engine
from sqlalchemy import text

engine = create_engine(
    "trino://trino@localhost:8080/hive", connect_args={"user": "trino"}
)

conn = engine.connect()
cur = conn.execute(text("CREATE SCHEMA IF NOT EXISTS public WITH (LOCATION='s3a://warehouse/notebooks/')"))

# https://pandas.pydata.org/docs/user_guide/io.html#insertion-method
dataframe.to_sql(
    con=engine,
    name="taxi",
    schema="public",
    method=insert_on_conflict_nothing,
    if_exists="replace",
    index=False,
)

OperationalError: (trino.exceptions.TrinoExternalError) TrinoExternalError(type=EXTERNAL, name=HIVE_PATH_ALREADY_EXISTS, message="Target directory for table 'public.taxi' already exists: s3a://warehouse/notebooks/taxi", query_id=20240216_085936_00067_c6ms9)
[SQL: 
CREATE TABLE public.taxi (
	sepal_length_cm DOUBLE, 
	sepal_width_cm DOUBLE, 
	petal_length_cm DOUBLE, 
	petal_width_cm DOUBLE, 
	species VARCHAR
)

]
(Background on this error at: https://sqlalche.me/e/20/e3q8)